**Run this notebook in Colab and Chrome!** (On github.com right-click the "Open in Colab" badge -> "open in new tab").

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/floleuerer/fastai-tutorials/blob/main/fastai_tensorboard_projector.ipynb)

In [ ]:
pip install -Uqq fastai

In [ ]:
from fastai.vision.all import *

# Tensorboard Projector with fastai

With fastai's TensorBoardProjectorCallback it's really easy to visualize **Image Embeddings** in Tensorboard Projector. There are two options:


1.   write image embeddings during training (use `TensorBoardCallback(projector=True)` during training)
2.   write image embeddings during inference e.g. to use a different dataset than the model was trained on (use `TensorBoardProjectorCallback()` during inference)

I'll show how to use TensorBoardProjectorCallback during inference!

For more information on the tensorboard-Integration see https://docs.fast.ai/callback.tensorboard.



## Train a model

We could just use a plain pretrained model to extract the image embeddings. But fine-tuning a model on the dataset or domain you are going to use, gives more meaningful features and embeddings.

In this tutorial we use the **Pets** dataset and fine-tune a **Resnet34** (see the [fastai docs](https://docs.fast.ai/tutorial.vision) for more information).

In [ ]:
path = untar_data(URLs.PETS)

In [ ]:
labeller = using_attr(RegexLabeller(pat = r'^(.*)_\d+.jpg$'), 'name')

blocks = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                 get_items=get_image_files, 
                 get_y=labeller,
                 splitter=RandomSplitter(valid_pct=0.2, seed=42),
                 item_tfms=Resize(224))

dls = blocks.dataloaders(path/'images')

In [ ]:
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)

In [ ]:
learn.fine_tune(3)

## Extracting Image Embeddings during Inference

We'll create a Dataloader for a subset of the Pets-dataset and run inference with the `TensorBoardProjectorCallback()`to write the image embeddings. Then we'll use **tensorboard** to visualize the embeddings.

In [ ]:
from fastai.callback.tensorboard import TensorBoardProjectorCallback
import random

In [ ]:
# only colab - workaround for bug https://github.com/PyTorchLightning/pytorch-lightning/issues/4214
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

Create the datloader for 256 images using `learn.dls.test_dl()`.

In [ ]:
files = get_image_files(path/'images')
random.shuffle(files)
files = files[:256]
dl = learn.dls.test_dl(files, with_labels=True)

Run `learn.get_preds()` and pass the dataloader and the `TensorBoardProjectorCallback()`.

In [ ]:
cbs = [TensorBoardProjectorCallback(log_dir='proj')]
_ = learn.get_preds(dl=dl, cbs=cbs)

## Visualize the Embeddings in Tensorboard Projector

Run tensorboard in the Notebook - Make sure to switch to **Projector** in the drop-down on the top-right corner. Du to colab's slow i/o the 'Fetching sprite image...' step can take a while.  You can switch between the different dimensional reduction algoritms on the left (PCA, t-SNE and UMAP).

![Projector](imgs/tb_projector.png)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /content/proj